In [2]:
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from core import criar_modelo, ParImageGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger

In [3]:
tf.config.run_functions_eagerly(True)

os.makedirs('../model', exist_ok=True)
os.makedirs('../results', exist_ok=True)

In [4]:
# Caminho para salvar o modelo principal
caminho_modelo = '../model/modelo_solda_resnet50.h5'

# Carregar ou criar o modelo
if os.path.exists(caminho_modelo):
    print("Carregando modelo salvo...")
    modelo = tf.keras.models.load_model(caminho_modelo)
else:
    print("Criando novo modelo...")
    modelo = criar_modelo()

Criando novo modelo...


In [5]:
# Recompilar com otimizador novo
modelo.compile(optimizer=tf.keras.optimizers.Adam(),
                loss='binary_crossentropy',
                metrics=['accuracy'])

In [6]:
# ============================================
# 🔁 Carregar dados e dividir entre treino/validação
# ============================================
pasta = '../image/treinamento'
if not os.path.exists(pasta):
    raise FileNotFoundError(f"Diretório {pasta} não encontrado.")
gen_temporario = ParImageGenerator(pasta, batch_size=1, augmentacao=False)
dados = list(zip(gen_temporario.imagens, gen_temporario.labels))

train_data, val_data = train_test_split(dados, test_size=0.2, shuffle=True)

# Criar geradores com dados separados
train_gen = ParImageGenerator(pasta, train_data, batch_size=8, augmentacao=True)
val_gen = ParImageGenerator(pasta, val_data, batch_size=8, augmentacao=False)

In [7]:
# ============================================
# 📦 Callbacks: checkpoint + logger
# ============================================
checkpoint = ModelCheckpoint(
    filepath='../model/melhor_modelo.h5',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

logger = CSVLogger('../results/historico_treinamento.csv', append=True)

In [8]:
# ============================================
# ▶️ Treinamento
# ============================================
modelo.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,
    steps_per_epoch=2,
    # steps_per_epoch=len(train_gen),
    callbacks=[checkpoint, logger],
    verbose=1
)

c:\Users\finan\OneDrive - Nimofast Brasil SA\Área de Trabalho\cod\python\AI\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
c:\Users\finan\OneDrive - Nimofast Brasil SA\Área de Trabalho\cod\python\AI\venv\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15s/step - accuracy: 0.3125 - loss: 0.7938 
Epoch 1: val_accuracy improved from -inf to 0.84821, saving model to ../model/melhor_modelo.h5


2/2 ━━━━━━━━━━━━━━━━━━━━ 96s 72s/step - accuracy: 0.3750 - loss: 0.7972 - val_accuracy: 0.8482 - val_loss: 4.6962
Epoch 2/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - accuracy: 0.9688 - loss: 0.2694 
Epoch 2: val_accuracy improved from 0.84821 to 0.85714, saving model to ../model/melhor_modelo.h5


2/2 ━━━━━━━━━━━━━━━━━━━━ 125s 96s/step - accuracy: 0.9583 - loss: 0.3574 - val_accuracy: 0.8571 - val_loss: 21.5419
Epoch 3/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.9688 - loss: 0.0640
Epoch 3: val_accuracy did not improve from 0.85714
2/2 ━━━━━━━━━━━━━━━━━━━━ 28s 22s/step - accuracy: 0.9583 - loss: 0.0832 - val_accuracy: 0.8571 - val_loss: 69.0150
Epoch 4/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.8750 - loss: 0.2291
Epoch 4: val_accuracy did not improve from 0.85714
2/2 ━━━━━━━━━━━━━━━━━━━━ 55s 49s/step - accuracy: 0.8750 - loss: 0.2304 - val_accuracy: 0.8482 - val_loss: 241.1167
Epoch 5/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.8750 - loss: 1.3717
Epoch 5: val_accuracy did not improve from 0.85714
2/2 ━━━━━━━━━━━━━━━━━━━━ 56s 48s/step - accuracy: 0.8750 - loss: 1.7535 - val_accuracy: 0.8482 - val_loss: 475.4986
Epoch 6/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - accuracy: 0.9688 - loss: 0.4675 
Epoch 6: val_accuracy did not improve from 0.85714
2/2 ━━━

In [9]:
# Avaliação final no conjunto de validação
loss, accuracy = modelo.evaluate(val_gen, verbose=1)
print(f"[AVALIAÇÃO FINAL] Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

# Salvar o modelo final
print("Salvando modelo final...")
modelo.save(caminho_modelo)

14/14 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.8593 - loss: 15782828.0000


[AVALIAÇÃO FINAL] Loss: 16016129.0000, Accuracy: 0.8571
Salvando modelo final...
